# Preprocessing 3
Investment Analysis with Machine Learning\
UROP 1100E (Man Yin Michael YEUNG, 2021 Fall)

In [1]:
import pandas as pd
import numpy as np
import re
import csv

In [2]:
cols = ['word']
for i in range(1,301): cols.append('X'+str(i))
vec_data = pd.read_csv('C:/UROP/full_version_data/sgns.financial.word',\
                       sep=' ', quoting=csv.QUOTE_NONE, encoding="utf8", skiprows=1, index_col=False, names=cols)

In [3]:
vec_data.index = vec_data.word
vec_data = vec_data.drop(columns=['word'])

In [4]:
vec_dict = vec_data[:100000].transpose().to_dict(orient='list')

In [5]:
text_data = pd.read_parquet('C:/UROP/full_version_data/preprocessed_data_1.parquet.gzip')

In [6]:
text_data.head()

,ID,SecuCode,date,specret,specret_2d,specret_tp2-tp6,jieba_content
0,181899,600323,2010-01-04,0.006658,-0.001522,-0.030300,"[南海发展nt, 水价n, 如期t, 上调v, 增厚v, 业绩n, 约d, 事项n, 公司n..."
1,181900,002024,2010-01-04,-0.004292,-0.022783,-0.047744,"[苏宁ns, 电器n, 内外s, 兼修v, 进入v, 新一轮nz, 增长期n, 投资vn, ..."
2,181901,002092,2010-01-04,0.030815,0.018962,0.055390,"[泰ns, 化学n, 发布v, 准东ns, 井田nr, 勘探vn, 报告n, 煤炭n, 开发..."
3,181939,002092,2010-01-04,0.030815,0.018962,0.055390,"[泰ns, 化学n, 打造v, 新疆ns, 资源n, 平台n, 公司n, 煤矿n, 开发v,..."
4,181902,000568,2010-01-04,-0.033701,-0.045938,-0.036113,"[泸州ns, 老窖n, 公司n, 长期d, 竞争力n, 值得v, 期待v, 白酒n, 行业n..."


In [7]:
vec_cols = []
for i in range(1,301): vec_cols.append('X'+str(i))

In [13]:
import ray

In [21]:
def get_chin(word): # remove POS
    return str(re.findall(r'[\u4e00-\u9fff]+', word))[2:-2]

In [22]:
def jbc2vec(jbc):
    jbc_vec = np.zeros(300)
    if len(jbc) == 0: return jbc_vec
    jbc = list(map(get_chin, jbc))
    for word in jbc: 
        if word in exist_words:
            jbc_vec += np.array(vec_dict[word])
    return list(jbc_vec / len(jbc))

In [18]:
exist_words = list(vec_dict.keys())

In [19]:
x_cols = []
for i in range(300): x_cols.append('X'+str(i+1))

In [ ]:
s = text_data['jieba_content'].copy().apply(jbc2vec)
temp_df = pd.DataFrame.from_dict(dict(zip(s.index, s.values))).T
temp_df.columns = x_cols
temp_df.to_parquet('all_vecs_raw.parquet.gzip')

In [ ]:
text_data.join(temp_df)

In [ ]:
df_col_str = 'text_data["X1"]'
for i in range(2,301): df_col_str += ',text_data["X{Num}"]'.format(Num=i)

In [ ]:
text_data['X_all'] = text_data['jieba_content'].parallel_apply(jbc2vec)

In [ ]:
exec('{Cols}=zip(*text_data["jieba_content"].map(jbc2vec))'.format(Cols=df_col_str))
text_data = text_data.drop(columns=['jieba_content'])
text_data.to_parquet('D:/michael/full_version_data/preprocessed_data_3.parquet.gzip')

In [11]:
seg_train = text_data[:100].copy()
seg_test = text_data[10000:10100].copy()

In [12]:
df_col_str_train = 'seg_train["X1"]'
for i in range(2,301): df_col_str_train += ',seg_train["X{Num}"]'.format(Num=i)

In [ ]:
exec('{Cols}=zip(*seg_train["jieba_content"].map(jbc2vec))'.format(Cols=df_col_str_train))

In [ ]:
exec('{Cols}=zip(*text_data["jieba_content"].map(jbc2vec))'.format(Cols=df_col_str_test))